# Process Piers' IDL files into a human-readable format

In [1]:
import numpy as np
import pandas as pd
import glob
from scipy.io.idl import readsav
from climateforcing.utils import mkdir_p

In [2]:
delta_N_cmip5 = {}
delta_T_cmip5 = {}

models = {}
nyears=156

for experiment in ['historical', 'historicalNat', 'historicalGHG']:
    delta_N_cmip5[experiment] = {}
    delta_T_cmip5[experiment] = {}
    models[experiment] = set()
    for variable in ['rsdt', 'rsut', 'rlut', 'tas']:
        file_list = (glob.glob('../data_input/cmip5_Forster_etal_2013/%s_*_%s_*.idlsave' % (variable, experiment)))
        for file in file_list:
            models[experiment].add(file.split('_')[5])
    models[experiment] = (list(models[experiment]))
    for model in models[experiment]:
        if model =='CCSM4':
            continue  # it's sick
        delta_N_cmip5[experiment][model] = {}
        delta_T_cmip5[experiment][model] = {}
        file_list = glob.glob('../data_input/cmip5_Forster_etal_2013/rsdt_%s_%s_*.idlsave' % (model, experiment))
        for rsdtfile in file_list:
            run = rsdtfile.split('_')[7][:-8]
            tasfile = rsdtfile.replace('rsdt', 'tas')
            rsutfile = rsdtfile.replace('rsdt', 'rsut')
            rlutfile = rsdtfile.replace('rsdt', 'rlut')
                   
            try:
                tasdata = readsav(tasfile)
                len_tas = len(tasdata['tas'])
                rsdtdata = readsav(rsdtfile)
                len_rsdt = len(rsdtdata['rsdt'])
                rsutdata = readsav(rsutfile)
                len_rsut = len(rsutdata['rsut'])
                rlutdata = readsav(rlutfile)
                len_rlut = len(rlutdata['rlut'])
                tasdata['tas']-tasdata['ctrl'][:len_tas]
                rsdtdata['rsdt']-rsdtdata['ctrl'][:len_tas]
                rsutdata['rsut']-rsutdata['ctrl'][:len_tas]
                rlutdata['rlut']-rlutdata['ctrl'][:len_tas]
            except:
                continue
            
            delta_T_cmip5[experiment][model][run] = np.ones(156) * np.nan
            delta_N_cmip5[experiment][model][run] = np.ones(156) * np.nan
            
            first_time = tasdata['time'][0]
            first_index = int(first_time - 1850)
            if model[:3] == 'Had':
                offset=-11
                first_index = first_index+1
            else:
                offset=0
            print(model, experiment, run, first_time, first_index)
            for i in range(first_index, 156):
                delta_T_cmip5[experiment][model][run][i] = np.mean(tasdata['tas'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(tasdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsdt = np.mean(rsdtdata['rsdt'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsdtdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rsut = np.mean(rsutdata['rsut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rsutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                rlut = np.mean(rlutdata['rlut'][(i-first_index)*12+offset:12+(i-first_index)*12+offset]) - np.mean(rlutdata['ctrl'][(i-first_index)*12+offset:12+(i-first_index)*12+offset])
                delta_N_cmip5[experiment][model][run][i] = rsdt - rsut - rlut
        delta_N_array = np.ones((nyears, len(delta_N_cmip5[experiment][model]))) * np.nan
        delta_T_array = np.ones((nyears, len(delta_T_cmip5[experiment][model]))) * np.nan
        for i, run in enumerate(delta_T_cmip5[experiment][model].keys()):
            delta_N_array[:, i] = delta_N_cmip5[experiment][model][run]
            delta_T_array[:, i] = delta_T_cmip5[experiment][model][run]
        delta_N_cmip5[experiment][model]['mean'] = np.nanmean(delta_N_array, axis=1)
        delta_T_cmip5[experiment][model]['mean'] = np.nanmean(delta_T_array, axis=1)

delta_N_cmip5['historicalOther'] = {}
delta_T_cmip5['historicalOther'] = {}
for model in delta_T_cmip5['historical'].keys():
    if (model not in delta_T_cmip5['historicalNat'].keys()) and (model not in delta_T_cmip5['historicalGHG'].keys()):
        continue
    print(model)
    delta_N_cmip5['historicalOther'][model] = {}
    delta_T_cmip5['historicalOther'][model] = {}
    for run in list(delta_T_cmip5['historical'][model].keys()):
        if (run not in delta_T_cmip5['historicalGHG'][model].keys()) or (run not in delta_T_cmip5['historicalNat'][model].keys()):
            continue
        delta_N_cmip5['historicalOther'][model][run] = delta_N_cmip5['historical'][model][run] - delta_N_cmip5['historicalGHG'][model][run] - delta_N_cmip5['historicalNat'][model][run]
        delta_T_cmip5['historicalOther'][model][run] = delta_T_cmip5['historical'][model][run] - delta_T_cmip5['historicalGHG'][model][run] - delta_T_cmip5['historicalNat'][model][run]
    delta_N_array = np.ones((156, len(delta_N_cmip5['historicalOther'][model]))) * np.nan
    delta_T_array = np.ones((156, len(delta_T_cmip5['historicalOther'][model]))) * np.nan
    for i, run in enumerate(delta_T_cmip5['historicalOther'][model].keys()):
        delta_N_array[:, i] = delta_N_cmip5['historicalOther'][model][run]
        delta_T_array[:, i] = delta_T_cmip5['historicalOther'][model][run]
    delta_N_cmip5['historicalOther'][model]['mean'] = np.mean(delta_N_array, axis=1)
    delta_T_cmip5['historicalOther'][model]['mean'] = np.mean(delta_T_array, axis=1)

C:\Users\smithc\AppData\Local\Temp/ipykernel_14104/3039339408.py:66: RuntimeWarning: Mean of empty slice
  delta_N_cmip5[experiment][model]['mean'] = np.nanmean(delta_N_array, axis=1)
C:\Users\smithc\AppData\Local\Temp/ipykernel_14104/3039339408.py:67: RuntimeWarning: Mean of empty slice
  delta_T_cmip5[experiment][model]['mean'] = np.nanmean(delta_T_array, axis=1)


MPI-ESM-LR historical r1i1p1 1850.0416 0
MPI-ESM-LR historical r3i1p1 1850.0416 0
MPI-ESM-LR historical r2i1p1 1850.0416 0
IPSL-CM5A-MR historical r1i1p1 1850.0416 0
ACCESS1-0 historical r1i1p1 1850.0416 0
GISS-E2-H historical r3i1p1 1850.0416 0
GISS-E2-H historical r1i1p1 1850.0416 0
GISS-E2-H historical r5i1p1 1850.0416 0
GISS-E2-H historical r4i1p1 1850.0416 0
GISS-E2-H historical r2i1p1 1850.0416 0
NorESM1-M historical r1i1p1 1850.0416 0
NorESM1-M historical r3i1p1 1850.0416 0
NorESM1-M historical r2i1p1 1850.0416 0
MPI-ESM-P historical r1i1p1 1850.0416 0
MPI-ESM-P historical r2i1p1 1850.0416 0
MIROC-ESM historical r2i1p1 1850.0416 0
MIROC-ESM historical r3i1p1 1850.0416 0
MIROC-ESM historical r1i1p1 1850.0416 0
NorESM1-ME historical r1i1p1 1850.0416 0
GFDL-ESM2G historical r1i1p1 1861.0416 11
GFDL-ESM2G historical r2i1p1 1861.0416 11
IPSL-CM5A-LR historical r2i1p1 1850.0416 0
IPSL-CM5A-LR historical r4i1p1 1850.0416 0
IPSL-CM5A-LR historical r6i1p1 1850.0416 0
IPSL-CM5A-LR histori

H:\conda\cmip5-cmip6-forcing\lib\site-packages\numpy\core\fromnumeric.py:3474: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
H:\conda\cmip5-cmip6-forcing\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
H:\conda\cmip5-cmip6-forcing\lib\site-packages\numpy\core\_methods.py:189: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


IPSL-CM5B-LR historical r1i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r4i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r9i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r6i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r10i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r2i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r1i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r3i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r5i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r7i1p1 1850.0416 0
CSIRO-Mk3-6-0 historical r8i1p1 1850.0416 0
inmcm4 historical r1i1p1 1850.0416 0
FGOALS-g2 historical r3i1p1 1850.0416 0
FGOALS-g2 historical r5i1p1 1850.0416 0
bcc-csm1-1 historical r2i1p1 1850.0416 0
bcc-csm1-1 historical r3i1p1 1850.0416 0
bcc-csm1-1 historical r1i1p1 1850.0416 0
CNRM-CM5 historical r1i1p1 1850.0416 0
CNRM-CM5 historical r3i1p1 1850.0416 0
CNRM-CM5 historical r5i1p1 1850.0416 0
CNRM-CM5 historical r10i1p1 1850.0416 0
CNRM-CM5 historical r8i1p1 1850.0416 0
CNRM-CM5 historical r7i1p1 1850.0416 0
CNRM-CM5 historical r4i1p1 1850.0416 0
CN

In [3]:
for expt in delta_N_cmip5:
    mkdir_p(f'../data_output/cmip5/{expt}')
    for model in delta_N_cmip5[expt]:
        mkdir_p(f'../data_output/cmip5/{expt}/{model}')
        for run in delta_N_cmip5[expt][model]:
            df_out = pd.DataFrame(
                {
                    'year': range(1850,2006),
                    'rndt': delta_N_cmip5[expt][model][run],
                    'tas': delta_T_cmip5[expt][model][run],
                }
            )
            df_out.to_csv(f'../data_output/cmip5/{expt}/{model}/{model}_historical_{run}.csv', index=False)